<a href="https://colab.research.google.com/github/mdkamrulhasan/gvsu_machine_learning/blob/main/notebooks/DataProcess_assigment_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

# Loading Data

In [4]:
sales = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/sales-data-set.csv")
features = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/Features-data-set.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/retail/stores-data-set.csv")

# Estimate monthly temperature at store locations (Year, Month level)


*   Given the temparatures are at the weekly-level



In [5]:
features.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [6]:
features['Date'] = pd.to_datetime(features.Date, format="%d/%m/%Y")
features['year'] = features.Date.dt.year
features['month'] = features.Date.dt.month

In [7]:
temperature_monthly = features.groupby(
    ['Store', 'year', 'month']).agg(
        {'Temperature': 'mean'}).rename(
            columns={'Temperature': 'monthly_temperature'}).reset_index()
temperature_monthly.head(2)

,Store,year,month,monthly_temperature
0,1,2010,2,41.845
1,1,2010,3,52.580


# Estimate monthly sales of stores (Year, Month level)


*   Given that the sales are at the weekly-level




In [8]:
sales.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True


In [9]:
sales['Date'] = pd.to_datetime(sales.Date, format="%d/%m/%Y")
sales['year'] = sales.Date.dt.year
sales['month'] = sales.Date.dt.month

In [10]:
# one additional step may be required: to check if we have duplicate entries per week??
sales_monthly = sales.groupby(
    ['Store', 'year', 'month']).agg(
        {'Weekly_Sales': 'sum'}).rename(
            columns={'Weekly_Sales': 'monthly_sales'}).reset_index()
sales_monthly.head(2)

,Store,year,month,monthly_sales
0,1,2010,2,6307344.10
1,1,2010,3,5871293.98


# Table joins

In [11]:
sales_temperature = sales_monthly.merge(temperature_monthly, on=['Store', 'year', 'month'], how='inner')
sales_temperature.head(2)

,Store,year,month,monthly_sales,monthly_temperature
0,1,2010,2,6307344.10,41.845
1,1,2010,3,5871293.98,52.580


In [12]:
stores.head(2)

,Store,Type,Size
0,1,A,151315
1,2,A,202307


# Final table

In [13]:
sales_temperature_store = sales_temperature.merge(stores, on=['Store'], how='inner')
sales_temperature_store.head(2)

,Store,year,month,monthly_sales,monthly_temperature,Type,Size
0,1,2010,2,6307344.10,41.845,A,151315
1,1,2010,3,5871293.98,52.580,A,151315


# Modeling

In [14]:
sales_temperature_store.head(2)

,Store,year,month,monthly_sales,monthly_temperature,Type,Size
0,1,2010,2,6307344.10,41.845,A,151315
1,1,2010,3,5871293.98,52.580,A,151315


Feature encoding

In [15]:
X_numeric = ['month', 'monthly_temperature', 'monthly_sales']
X_categorical = ['Type']


# 1-Hot encode categorical data
# you can also use label-encoder with certain caution

one_hot_dataframes = []
for featx in X_categorical:
  one_hot_dataframes.append(
      pd.get_dummies(sales_temperature_store[featx], prefix=featx)
  )

In [16]:
data = pd.concat([sales_temperature_store[X_numeric]]+ one_hot_dataframes, axis=1)
data.head(2)

,month,monthly_temperature,monthly_sales,Type_A,Type_B,Type_C
0,2,41.845,6307344.10,1,0,0
1,3,52.580,5871293.98,1,0,0


Separating features and labels

In [19]:
features = ['month', 'monthly_temperature', 'Type_A', 'Type_B', 'Type_C']
label = 'monthly_sales'
X = data[features]
y = data[label]

Feature scaling

In [20]:
# Normalize input data through min-max scaling
feature_scaler = MinMaxScaler()
X = feature_scaler.fit_transform(np.asarray(X))

Data splitting

In [21]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.20, random_state=42)

Model instantiation and training

In [22]:
# model instantiation
regr = LinearRegression()
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression()

Training error

In [23]:
# Mean absolute error (in millions)
mse = mean_absolute_error(y_train, regr.predict(X_train))/1000000
print("Mean squared error: %.2fM" % mse)

Mean squared error: 1.45M


Test error

In [27]:
# Mean absolute error (in millions)
y_pred = regr.predict(X_test)
mse = mean_absolute_error(y_test, y_pred)/1000000
print("Mean squared error: %.2fM" % mse)

Mean squared error: 1.47M


# Plotting labels vs predictions

In [49]:
fig = go.Figure([
    go.Scatter(x=np.arange(len(y_test)), y=y_test, name='sales(ground-truth)'),
    go.Scatter(x=np.arange(len(y_test)), y=y_pred, name='predicted-sales')
]
               )

fig.update_layout(
    title="LR (monthly sales prediction)", yaxis_title="monthly sales", legend_title="")
fig.update_layout(
    legend=dict(
        x=0.8,
        y=0.95
    )
)
fig.show()